In [16]:
# Setup 
import time
import numpy as np
import matplotlib.pyplot as plt
import generate_image_files as gif
from cs231n.classifiers.fc_net import *
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.solver import Solver


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# Load images
data = gif.get_data()
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']
               
# Normalize the data: subtract the mean image
mean_image = np.mean(X_train)
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image

# Print data size and shape
for k, v in data.iteritems():
  print '%s: ' % k, v.shape


data #1, emotion class:0, category:Training

data #2, emotion class:0, category:Training

data #3, emotion class:2, category:Training

data #4, emotion class:4, category:Training

data #5, emotion class:6, category:Training

data #6, emotion class:2, category:Training

data #7, emotion class:4, category:Training

data #8, emotion class:3, category:Training

data #9, emotion class:3, category:Training

data #10, emotion class:2, category:Training

data #11, emotion class:0, category:Training

data #12, emotion class:6, category:Training

data #13, emotion class:6, category:Training

data #14, emotion class:6, category:Training

data #15, emotion class:3, category:Training

data #16, emotion class:5, category:Training

data #17, emotion class:3, category:Training

data #18, emotion class:2, category:Training

data #19, emotion class:6, category:Training

data #20, emotion class:4, category:Training

data #21, emotion class:4, category:Training

data #22, emotion class:2, category:Trainin

KeyboardInterrupt: 

In [ ]:
# 
from cs231n.classifiers.convnet import *
convlayer_params = {'convlayer_num': 2 ,'num_filters': [32, 32], 'filter_size': 3,
                   'stride': 1, 'pad':(3 - 1) / 2, 's_batch_norm': False,
                   'pool_dim': 2, 'pool_stride': 2}
affinelayer_params = {'afflayer_num': 1, 'hidden_dim': [50], 'batch_norm': False,
                      'dropout': False}

print 'check initial loss:'
###Loss check###
weight_scales = 5e-2
model = MyFullyConvNet(convlayer_params, affinelayer_params, weight_scale=weight_scales, reg=0.0)

X = np.random.randn(100, 1, 48, 48)
y = np.random.randint(7, size=100)
loss, _ = model.loss(X, y)

# Sanity check: Loss should be about log(10) = 2.3026
print 'Sanity check: Initial loss (no regularization): ', loss

# Sanity check: Loss should increase when we add regularization
model = MyFullyConvNet(convlayer_params, affinelayer_params, weight_scale=weight_scales, reg=0.01)
loss, _ = model.loss(X, y)
print 'Sanity check: Initial loss (with regularization): ', loss
print ' '
print 'check gradient:'

###Gradient check using numerical gradient computatation###
num_inputs = 2
input_shape = (1, 16, 16)
reg = 0.0
num_classes = 7
X = np.random.randn(num_inputs, *input_shape)
y = np.random.randint(num_classes, size=num_inputs)

convlayer_params = {'convlayer_num': 2 ,'num_filters': [3, 3], 'filter_size': 3,
                   'stride': 1, 'pad':(3 - 1) / 2, 's_batch_norm': False,
                   'pool_dim': 2, 'pool_stride': 2}

affinelayer_params = {'afflayer_num': 1, 'hidden_dim': [7], 'batch_norm': False,
                      'dropout': False}

model = MyFullyConvNet(convlayer_params, affinelayer_params, weight_scale=weight_scales,
                       reg=reg, input_dim=input_shape,
                       num_classes=num_classes, dtype=np.float64)


loss, grads = model.loss(X, y)
for name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    param_grad_num = eval_numerical_gradient(f, model.params[name], verbose=False, h=1e-6)
    e = rel_error(param_grad_num, grads[name])
    print '%s max relative error: %e' % (name, rel_error(param_grad_num, grads[name]))

print ' '
print 'Over fit a small data:'
###Overfit small data###
num_train = 100
small_data = {
  'X_train': X_train[:num_train],
  'y_train': y_train[:num_train],
  'X_val': X_val,
  'y_val': y_val,
}

convlayer_params = {'convlayer_num': 2 ,'num_filters': [32, 32], 'filter_size': 3,
                   'stride': 1, 'pad':(3 - 1) / 2, 's_batch_norm': False,
                   'pool_dim': 2, 'pool_stride': 2}
affinelayer_params = {'afflayer_num': 1, 'hidden_dim': [50], 'batch_norm': False,
                      'dropout': False}

model = MyFullyConvNet(convlayer_params, affinelayer_params, weight_scale=weight_scales, reg=0.0)

solver = Solver(model, small_data,
                num_epochs=20, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-3,
                },
                verbose=True, print_every=1)
solver.train()

print ' '
print' Lets train the real Net Kaggle data:'
###AND FINALLY TRAIN WITH ALL CIFAR-10 DATA!###
#Below you can find the best model parameters that I found to train the net.#
# The accuracies as well as the loss and accuracy histories are plotted in the next box.#

convlayer_params = {'convlayer_num': 2 ,'num_filters': [32, 64], 'filter_size': 3,
                   'stride': 1, 'pad':(3 - 1) / 2, 's_batch_norm': False,
                   'pool_dim': 2, 'pool_stride': 2}
affinelayer_params = {'afflayer_num': 1, 'hidden_dim': [100], 'batch_norm': False,
                      'dropout': False}

reg = 0.0001
weight_scales = 5e-2
learn_rate = 1e-3
model = MyFullyConvNet(convlayer_params, affinelayer_params, weight_scale=weight_scales, reg=reg)

solver = Solver(model, data,
                num_epochs=20, batch_size=100,
                update_rule='adam',
                optim_config={
                'learning_rate': learn_rate,},
                  verbose=True, print_every=100)
solver.train()
